# Create a forge and execute an evolution cycle
In this notebook, we will go through the very few steps needed to run a forge cycle for a given budget. 
Before running this notebook, you must set up a few things.

## Setup steps

### 1. Your access to LLMs:

To try Ebiose, either you have a valid Ebiose's API key, or you have your own LLM API keys. 
In both situations, you must first copy-paste the [`model_endpoints_template.yml`](./../model_endpoints_template.yml) YAML file, rename it as `model_endpoints.yml` 
and fill it with the credentials relative to your situation.

#### 1.1. With an Ebiose API key

With a valid Ebiose API key, you just have to set the `ebiose.api_key` field
with the API key you can get on [https://app.ebiose.com/](https://app.ebiose.com/).
Don't modify any field in this YAML file.

#### 1.2. With a LiteLLM account or self-proxy

If you are using LiteLLM, either through LiteLLM cloud or a self-hosted proxy,
You must fill the `lite_llm` field with your API key and LiteLLM URL.

#### 1.3. With other providers

In this case, you have to fill the `endpoints` field with the complete
credentials to your own LLM provider.

### 2. Add the root directory to your Python path

Depending on your settings, you may need to add the root of the repository to your `PYTHONPATH` environment variable. You may also use a `.env` file to do so. copy-paste the [`.env.template`](./../.env.template) file, rename it as `.env` and fill it with your own root directory.

### 3. Use LangFuse for tracing (optional)

Ebiose implements LangFuse to provide easy and free observability, through its self-hosted capability. Refer to [Langfuse official documentation](https://langfuse.com/self-hosting) to set it up. Once done, fill `LANGFUSE_SECRET_KEY`, `LANGFUSE_PUBLIC_KEY` and `LANGFUSE_HOST` in the `.env` file.

Other observability tools might be used but are not configured yet.

### 4. Load the .env file

To load the `.env` file, execute:

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## Creating a basic forge

In ebiose, a **forge** is where custom agents are created to solve specific problems. The forge is the exclusive origin of new agents. Within each forge, architects agents orchestrate the creation and improvement of agents by reusing existing building blocks from the ecosystem.

To create a forge and run a cycle, you must provide the following:
- a description of the forge, which defines the problem that must be solved by generated agents;
- the expected format of the agent's input and output, defined as Pydantic models;
- an implementation of the `compute_fitness` abstract method that will be used by the forge to evaluate the generated agents.

Let's say we wich to generate agents specialized in solving math problems. The forge description could be:

In [2]:
forge_description = "Solving math word problems"

Next, we need to define the expected input and output formats of the generated agents. These formats are to be defined as Pydantic models. 

For instance, in our context of solving math problems, we want the agent input to be a string which will represent the math problem to be solved and the agent output to be composed of two fields:
- `solution` which will be the final solution to the math problem, given as an integer;
- `rationale` which will be the rationale behind the found solution.

The IO Pydantic models will thus be:

In [3]:
from pydantic import BaseModel

class AgentInput(BaseModel):
        math_problem: str

class AgentOutput(BaseModel):
    solution: int
    rationale: str

Lastly, we must provide a way of evaluating the generated agents through the implementation of the `compute_fitness` abstract method of `AgentForge` class. For the sake of demonstration, we will here return a random float between 0 and 1, so that we don't spend tokens at evaluation.

In [4]:
import random
random.seed(11)

from ebiose.core.agent import Agent
from ebiose.core.agent_forge import AgentForge

class BasicForge(AgentForge):
    async def compute_fitness(self, agent: Agent, **kwargs: dict[str, any]) -> float:
        return (agent.id, random.random())

We can now instantiate the forge with the provided elements:

In [5]:
forge = BasicForge(
    name="Basic forge",
    description=forge_description,
    agent_input_model=AgentInput,
    agent_output_model=AgentOutput,
    default_generated_agent_engine_type="langgraph_engine",
    default_model_endpoint_id=None, # set to a different model endpoint id 
    # if you want to use a specific model for generated agents, 
    # other than the default one defined in the model endpoint YAML file
)

## Setting up a forge cycle

Before running a forge cycle, you must set it up with a few parameters. 
The setup depends on whether you have an Ebiose API key or not.

If you have an Ebiose API key, set the `MODE` variable to `"cloud"`. Otherwise, 
set it up to `"local"`.

In any case, you must, at least, set up `N_AGENTS_IN_POPULATION` the number of agents in each generation.

If you are in `Mode=="cloud"`, you must also set up the `BUDGET` variable which 
represents the maximum budget allocated to run the forge cyle. Otherwise, for `Mode=="local"`, you must setup the `N_GENERATIONS` variable.

In [6]:
from ebiose.core.forge_cycle import CloudForgeCycleConfig, LocalForgeCycleConfig

MODE = "cloud" # or "local" for local mode
N_AGENTS_IN_POPULATION = 2 # number of agents in the population, at each generation
BUDGET = 0.01 # the budget for the forge cycle in dollars
N_GENERATIONS = 1 # number of generations in the forge cycle
if MODE == "cloud":
    cycle_config = CloudForgeCycleConfig(
        budget=BUDGET,
        n_agents_in_population=N_AGENTS_IN_POPULATION,
        n_selected_agents_from_ecosystem=1,
        n_best_agents_to_return=2,
        replacement_ratio=0.5,
        node_types=["StartNode", "EndNode", "LLMNode"],
    )
elif MODE == "local":
    cycle_config = LocalForgeCycleConfig(
        n_generations = N_GENERATIONS,
        n_agents_in_population=N_AGENTS_IN_POPULATION,
        n_selected_agents_from_ecosystem=0,
        n_best_agents_to_return=2,
        replacement_ratio=0.5,
        node_types=["StartNode", "EndNode", "LLMNode" ],
    )


## Running a forge cycle

Once the forge is set up, we can start generating agents by running a **forge cycle**. 

> ⚠️ Note that we need to use `asyncio.run` to launch the forge cycle.

> 🚨 Before executing the following cell, double check the previous cell which defined the forge cycle configuration!

> 💡 If you are using VSCode, install the [*Markdown Preview Mermaid Support* extension](https://marketplace.visualstudio.com/items?itemName=bierner.markdown-mermaid) to allow the display of the generated agent's graphs.

In [7]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

# Run the forge cycle
best_agents, best_fitness = asyncio.run(
    forge.run_new_cycle(config=cycle_config)
)



Attempting to get first ecosystem uuid...

Attempting to get ecosystems...

Attempting to get ecosystem...


INFO: Model 'InputModel' created and rebuilt successfully.
INFO: Final rebuild completed for top-level model 'InputModel'.
INFO: Model 'Edge' created and rebuilt successfully.
INFO: Model 'EndNode' created and rebuilt successfully.
INFO: Model 'LLMNode' created and rebuilt successfully.
INFO: Model 'StartNode' created and rebuilt successfully.
INFO: Model 'RoutingNode' created and rebuilt successfully.
INFO: Model 'PydanticValidatorNode' created and rebuilt successfully.
INFO: Model 'OutputModel' created and rebuilt successfully.
INFO: Final rebuild completed for top-level model 'OutputModel'.
INFO: Model 'InputModel' created and rebuilt successfully.
INFO: Final rebuild completed for top-level model 'InputModel'.
INFO: Model 'Edge' created and rebuilt successfully.
INFO: Model 'EndNode' created and rebuilt successfully.
INFO: Model 'LLMNode' created and rebuilt successfully.
INFO: Model 'StartNode' created and rebuilt successfully.
INFO: Model 'RoutingNode' created and rebuilt success


Attempting to start new forge cycle...

Attempting to add forge...


Event: ForgeCycleStartedEvent 
 {
  "timestamp": "2025-06-20T23:37:11.564902Z",
  "forge_name": "Basic forge",
  "forge_description": "Solving math word problems",
  "config": {
    "n_agents_in_population": 2,
    "n_selected_agents_from_ecosystem": 1,
    "n_best_agents_to_return": 2,
    "replacement_ratio": 0.5,
    "tournament_size_ratio": 0.1,
    "local_results_path": null,
    "mode": "cloud",
    "budget": 0.01
  },
  "event_name": "ForgeCycleStartedEvent"
}
****** Initializing agents population ******
Event: PopulationInitializationStartedEvent 
 {
  "timestamp": "2025-06-20T23:37:11.566086Z",
  "n_agents_to_initialize": 2,
  "n_selected_from_ecosystem": 1,
  "event_name": "PopulationInitializationStartedEvent"
}



Attempting to get user id...

Attempting to select agents...


...


Attempting to get cost...


  0%|          | 0/1 [00:00<?, ?it/s]
Initializing structured output agent for model OutputModel (2)
INFO: Model 'InputModel' created and rebuilt successfully.
INFO: Final rebuild completed for top-level model 'InputModel'.
INFO: Model 'OutputModel' created and rebuilt successfully.
INFO: Final rebuild completed for top-level model 'OutputModel'.
100%|██████████| 1/1 [00:34<00:00, 34.08s/it]
Event: OffspringCreatedEvent 
 {
  "timestamp": "2025-06-20T23:40:02.337995Z",
  "offspring_agent_id": "agent-c1af8ede-5657-4c95-9ee1-783d34a0a6cf",
  "generation_number": 0,
  "parent_ids": [
    "agent-96b234ad-9ee2-4eb0-8459-0d10cd618ccb"
  ],
  "genetic_operator_agent_id": "agent-dff043b2-fa0f-4761-a1f0-5561bd58ed26",
  "offspring_agent": {
    "id": "agent-c1af8ede-5657-4c95-9ee1-783d34a0a6cf",
    "name": "Solving math word problems",
    "agent_type": null,
    "description": "",
    "architect_agent_id": "agent-c6ef9d53-923b-459a-ab99-ebb2877517d8",
    "genetic_operator_agent_id": "agent-d

Agent engine run completed.
Agent engine run completed.

Attempting to get cost...

Attempting to add agent from forge cycle...


Event: AgentAddedToPopulationEvent 
 {
  "timestamp": "2025-06-20T23:40:02.606893Z",
  "agent_id": "agent-c79c160b-1d88-4d4c-b9bb-4e9ae2ca7e6a",
  "generation_number": 0,
  "source": "offspring",
  "agent": {
    "id": "agent-c79c160b-1d88-4d4c-b9bb-4e9ae2ca7e6a",
    "name": "Solving math word problems",
    "agent_type": null,
    "description": "",
    "architect_agent_id": "agent-c6ef9d53-923b-459a-ab99-ebb2877517d8",
    "genetic_operator_agent_id": "agent-dff043b2-fa0f-4761-a1f0-5561bd58ed26",
    "architect_agent": null,
    "parent_ids": [
      "agent-96b234ad-9ee2-4eb0-8459-0d10cd618ccb"
    ],
    "agent_engine": {
      "engine_type": "langgraph_engine",
      "configuration": "{\"input_model\": {\"properties\": {\"math_problem\": {\"title\": \"Math Problem\", \"type\": \"string\"}}, \"required\": [\"math_problem\"], \"title\": \"InputModel\", \"type\": \"object\"}, \"output_model\": {\"properties\": {\"solution\": {\"title\": \"Solution\", \"type\": \"integer\"}, \"rationa


Attempting to get cost...

Attempting to end forge cycle...
An unexpected error occurred: 1 validation error for AgentInputModel
agentType
  Field required [type=missing, input_value={'uuid': 'agent-c79c160b-...iption_embedding': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing


We can now display the best agents that have been returned as follows. Note that:
- all agents can be found in the `SAVE_PATH` directory if you defined one;
- here, the compute fitness only returns a random float, so the following displayed agents have not been truly evaluated. 

Go check [examples/math_forge/math_forge.py](./../examples/math_forge/math_forge.py) to see a fully implemeted forge with a non-random fitness evaluation function.

## Display best agents

In [8]:
forge.display_results(best_agents, best_fitness)

# Agent ID: agent-c79c160b-1d88-4d4c-b9bb-4e9ae2ca7e6a
## Fitness: 0.842427128518532
```mermaid 
graph LR
	Start_Node[start_node] --> Problem_Understanding_Node(Problem Understanding)
	Problem_Understanding_Node(Problem Understanding) --> Initial_Calculation_Node(Initial Calculation)
	Initial_Calculation_Node(Initial Calculation) --> Validation_Of_Solution_Node(Validation of Solution)
	Validation_Of_Solution_Node(Validation of Solution) -->|if validation fails| Refinement_Of_Approach_Node(Refinement of Approach)
	Validation_Of_Solution_Node(Validation of Solution) -->|if validation succeeds| Final_Solution_Presentation_Node(Final Solution Presentation)
	Validation_Of_Solution_Node(Validation of Solution) -->|if further exploration is needed| Alternative_Solution_Node(Alternative Solution Exploration)
	Alternative_Solution_Node(Alternative Solution Exploration) --> Final_Solution_Presentation_Node(Final Solution Presentation)
	Refinement_Of_Approach_Node(Refinement of Approach) --> Final_Solution_Presentation_Node(Final Solution Presentation)
	Final_Solution_Presentation_Node(Final Solution Presentation) --> End_Node[end_node]
	Validation_Of_Solution_Node(Validation of Solution) -->|if insights on problem-solving strategies are needed| Additional_Insights_Node(Additional Insights Exploration)
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to solve word math problems through a series of interconnected reasoning stages. Each LLM node has a specific role in this process. As you engage with the task, focus on providing clear and detailed outputs that will guide the next node in the chain. Emphasize chain-of-thought reasoning, self-reflection, and evaluative feedback as you navigate this problem-solving journey. Your contributions are vital in building up the final solution.
##### Problem Understanding
Analyze the provided word math problem in detail. Extract and identify all key numerical data and relationships present in the problem statement. Explain the problem in your own words, detailing the operations that may be required. Highlight any ambiguities or assumptions you might encounter, and suggest potential implications of these uncertainties. Your output should summarize the main components of the problem and propose potential strategies for solving it.
##### Initial Calculation
Utilizing the information extracted from the previous node, perform the initial calculations necessary to solve the word math problem. Clearly outline each step of your calculation process and explain the rationale for each action taken. If you encounter unexpected results or uncertainties, reflect on your approach and provide a detailed rationale for your method. Consider alternative methods of calculation if necessary.
##### Validation of Solution
Review the calculations provided by the previous node. Validate the results against the original problem statement and ensure they logically follow from the operations performed. Identify any potential errors in reasoning or calculation, and suggest alternative approaches or corrections. Based on the validation, decide whether to forward the results to the final presentation node, refine the approach, explore alternative solutions, or provide additional insights.
##### Refinement of Approach
If the previous node identified issues with the calculations, propose a refined approach to the word math problem. Review the identified errors and suggest corrections or alternative strategies that could lead to a valid solution. Clearly outline the new steps you would take, detailing the reasoning behind each modification. Present this revised approach to the next node for final solution presentation.
##### Alternative Solution Exploration
If the validation suggests the need for further exploration, examine the problem from different angles. Investigate alternative methods or perspectives that could lead to a valid solution. Present your findings and reasoning to the next node for final solution presentation. Emphasize innovative approaches and any unique insights.
##### Final Solution Presentation
Consolidate the validated calculations or refined solutions from the previous nodes. Present the final answer to the word math problem in a clear and comprehensible format. Include a summary of the steps taken to reach this solution, highlighting any critical reasoning or decision-making processes. Ensure that the final output is ready for submission, providing clarity for anyone reviewing the solution.
##### Additional Insights Exploration
Based on the validation of the solution and the exploration of alternatives, provide deeper insights into the problem-solving process. Suggest additional strategies, methodologies, or considerations that could enhance the understanding or solution of the word math problem. Discuss the implications of these insights for future problem-solving.

# Agent ID: agent-68537980-c2dc-455d-9815-228502ab4d75
## Fitness: 0.4405311166566568
```mermaid 
graph LR
	Start_Node[StartNode] --> Problemunderstanding(Problem Understanding)
	Problemunderstanding(Problem Understanding) --> Dataextraction(Data Extraction)
	Dataextraction(Data Extraction) --> Equationformulation(Equation Formulation)
	Equationformulation(Equation Formulation) --> Initialcalculation(Initial Calculation)
	Initialcalculation(Initial Calculation) --> Solutionverification(Solution Verification)
	Solutionverification(Solution Verification) -->|if verification fails| Alternativeapproaches(Alternative Approaches)
	Solutionverification(Solution Verification) -->|if verification succeeds| Finalcalculation(Final Calculation)
	Alternativeapproaches(Alternative Approaches) --> Finalcalculation(Final Calculation)
	Finalcalculation(Final Calculation) --> Formattingtheanswer(Formatting the Answer)
	Formattingtheanswer(Formatting the Answer) --> Reflectionandfeedback(Reflection and Feedback)
	Reflectionandfeedback(Reflection and Feedback) --> End_Node[EndNode]
 
``` 
## Prompts:
##### Shared context prompt
This graph represents a collaborative problem-solving network designed to tackle mathematical word problems. Each node has a specific role in interpreting, analyzing, and solving the problem. As an LLM in this network, your task is to perform your designated function effectively, communicate relevant insights to the next node, and contribute to the overall solution. Utilize chain-of-thought reasoning, self-reflection, and evaluative feedback where applicable to enhance your responses.
##### Problem Understanding
Your role is to analyze the provided math word problem. Carefully read the problem and extract the essential elements, including the main question, relevant context, and any key details. Summarize the problem in your own words to ensure clarity. Convey this understanding to the next node, highlighting the main components that need to be addressed in the problem-solving process.
##### Data Extraction
Based on the summary provided by LLMNode 1, identify and list all relevant numerical data and variables present in the problem. This includes quantities, relationships, and any mathematical operations implied by the text. Clearly organize this information and prepare it for the next node, ensuring it is easy to interpret and use in the formulation of equations.
##### Equation Formulation
Utilizing the extracted data from LLMNode 2, formulate the appropriate mathematical equation(s) that represent the relationships described in the problem. Explain your reasoning and the steps you took to derive the equations. Provide these equations to the next node along with any assumptions made during the formulation process.
##### Initial Calculation
Using the equations provided by LLMNode 3, perform the initial calculations necessary to arrive at a preliminary solution. Clearly show your calculations step by step, and include any intermediate results. If the calculation yields multiple potential solutions or scenarios, present them all concisely. Pass these results to the next node for verification.
##### Solution Verification
Review the calculations performed by LLMNode 4. Check for accuracy and ensure that the results are consistent with the mathematical principles involved. If the solution appears valid, summarize your verification process. If there are errors or inconsistencies, identify them and provide corrective feedback to LLMNode 4, suggesting necessary adjustments. Forward either the verified solution or the feedback for revision to the next node.
##### Alternative Approaches
Reflect on the verification results from LLMNode 5. If the original calculations were successful, consider alternative methods or strategies that could be used to solve the problem. Present these alternatives clearly, along with their potential advantages or drawbacks. If there were issues with the initial solution, suggest alternative approaches to address and resolve those problems. Convey this information to LLMNode 7 for final calculations.
##### Final Calculation
Based on the verified solution or alternative approaches provided by LLMNode 6, perform any necessary final calculations or adjustments. Ensure that your calculations align with the problem's requirements and address any feedback received. Present the final answer in a clear and concise format, preparing it for presentation to the next node.
##### Formatting the Answer
Take the final answer from LLMNode 7 and format it appropriately for presentation. Ensure that the answer is clear, comprehensive, and includes any necessary units or contextual information. If applicable, provide a brief explanation of how the solution was derived, so it is easily understandable. Pass this formatted answer to LLMNode 9 for reflection.
##### Reflection and Feedback
Reflect on the entire problem-solving process, considering the contributions of each node. Evaluate the strengths and weaknesses of the approach taken. Provide constructive feedback on potential improvements for future problem-solving scenarios, including common pitfalls and insights gained. Summarize your reflections and feedback, and present this to the EndNode as a concluding insight into the problem-solving journey.

